In [1]:
import requests
import pandas as pd
import json
import numpy as np
import pickle
from sklearn import linear_model

In [2]:
def extractCRData(scraped):
    l=[]
    for a in scraped:
        d={}
        d["id"]=a["model"]["id"]
        d["vin"]=a["vin"]
        d["make"]=a["make"]["name"]
        d["model"]=a["model"]["name"]
        d["testCompletionDate"]=a["testCompletionDate"]
        d["tiresAsTested"]=a.get("tiresAsTested", "None")
        d["fullTrimName"]=a.get("trimName", "None")
        d["autonomous"]=a["trim"]["modelYear"].get("autonomous", "None")
        d["modelYear"]=a["trim"]["modelYear"].get("year", "None")
        d["isRecommended"]=a.get("isRecommended", "None")
        d["isRedesign"]=a["trim"]["modelYear"].get("isRedesign", "None")
        d["transmission"]=a["trim"]["transmission"].get("description", "None")
        d["ratingsCategory"]=a.get("ratingsCategory", "None")
        d["trim"]=a["trim"]["name"]
        d["overallScore"]=a.get("crOverallScore", "None")
        d["roadTestScore"]=a.get("overallScore", "None")
        d["purchasePrice"]=a.get("purchasePrice")
    
        for item in a["trim"]['featureGroups']:
                for attribute in item['attributes']:
                    d[attribute['name']]=attribute['value']
                
        for item in a["testResults"]:
                d[item['name']]=item.get("overallRatingBlob", "None")
                if "subTests" in item:
                    for subtest in item["subTests"]:
                        d[subtest["name"]]=subtest.get("averageValue", "None")
                        
        l.append(d)

    df=pandas.DataFrame(l)
    #df=df.join(df['Luggage'].str.split('+', 1, expand=True).rename(columns={0:'duffel', 1:'suitcase'})).drop("Luggage",axis=1)
    
    return df

In [ ]:
apiLinkFront=""
apiLinkBack=""

In [ ]:
a=[]
with open('skipList.pkl', 'rb') as fp:
    skipList=pickle.load(fp)

for i in range(83,1001):
    if i in skipList:
        continue
    try:
        response = requests.get("".join([apiLinkFront,str(i),apiLinkBack]))
        response.raise_for_status() # raise exception if invalid response
        a.extend(response.json())
        print("Successfully visited {0}".format(str(i)))
    except requests.exceptions.RequestException as e:
        #print(e)
        skipList.append(i)
        continue



In [ ]:
with open('scrapelists.pkl', 'wb') as fp:
    pickle.dump([a,skipList], fp)

In [3]:
with open('scrapelists.pkl', 'rb') as fp:
    a, skipList=pickle.load(fp)

In [4]:
df=extractCRData(item)

In [6]:
l=["roadTestScore","Luggage","ratingsCategory"]
for i in a[0]['testResults']:
    #l.append(i["name"])
    if 'subTests' in i:
        for j in i['subTests']:
            l.append(j['name'])

In [7]:
dropList=['Quarter Mile Mph','Automatic Transmission Shift Action','Trunk Space','Brake Pedal Effort 1st Stop','Brake Pedal Effort 10th Stop', 'Cutoff', 'Gap', 'High Beams', 'Intensity', 'Low Beams', 'Opposing', 'Uniformity', 'Veiling', 'Width','0-100 Km/H','0-50 Km/H','500 M, Sec','80-100 Km/H','Noise 40 Mph Rough', 'Noise 70 Mph Smooth','City Traffic Cycle (L/Km)', 'City Traffic Cycle (Mpg)', 'Constant Speed 65 Mph (L/Km)', 'Constant Speed 65 Mph (Mpg)', 'Day Trip (Mpg)', 'Gallons/12000 Miles (Gal)', 'Overall (L/Km)','Noise Wot']
df_tests=df[l].drop(dropList,axis=1)

In [8]:
print(df_tests.columns.tolist())

['roadTestScore', 'Luggage', 'ratingsCategory', '0-30 Mph', '0-60 Mph', '45-65 Mph', 'Quarter Mile,Sec', 'Shifting Smoothness', 'Overall Routine Handling', 'Turning Circle', 'Avoidance Confidence', 'Avoidance Speed', 'Dry Skid Lateralg', 'Predictability Controllability', 'Steering Feel', 'Brake Fade', 'Brake Modulation', 'Dry Braking Directional Stability', 'No-Lock Stopping Distance', 'Split Mu Braking Directional Stability', 'Split Mu Stopping Distance', 'Wet Braking Stopping Distance', 'Normal Load Highway', 'Normal Load Rough Road1', 'Normal Load Rough Road2', 'Noise Trip Overall', 'Driving Position For Medium', 'Driving Position For Short', 'Driving Position For Tall', 'Driving Visibility For Medium', 'Driving Visibility For Short', 'Driving Visibility For Tall', 'Front Seat Comfort For Medium', 'Front Seat Comfort For Short', 'Front Seat Comfort For Tall', 'Rear Seat Comfort With 2', 'Climate Control Overall', 'Front Access For Medium', 'Front Access For Short', 'Front Access For

In [9]:
df_tests=df_tests.join(df_tests['Luggage'].str.split('+', 1, expand=True).rename(columns={0:'duffel', 1:'suitcase'})).drop("Luggage",axis=1)

In [10]:
df_tests = df_tests[df_tests['roadTestScore']!='None'].reset_index(drop='True')
df_tests.head()

,roadTestScore,ratingsCategory,0-30 Mph,0-60 Mph,45-65 Mph,"Quarter Mile,Sec",Shifting Smoothness,Overall Routine Handling,Turning Circle,Avoidance Confidence,...,Controls And Displays For Tall,Merp,Points Off,Overall (Mpg),Interior Fabric,Other Cockpit,Primary Cockpit,Trunc,duffel,suitcase
0,81.58,"Sports/sporty cars under $40,000",2.68,6.58,4.48,15.15,4.60,4.47,37.17,4.15,...,5.00,24.13,-1.1,28.56,3.50,3.80,3.80,3.7,2,1
1,85.78,Luxury compact cars,2.80,6.87,4.73,15.24,4.74,4.58,39.50,4.20,...,NaN,NaN,NaN,25.67,4.50,4.40,4.50,4.0,2,2
2,86.18,Luxury compact cars,3.45,8.53,5.10,16.75,4.70,4.60,38.17,4.30,...,2.77,NaN,-0.6,35.30,4.53,4.43,4.53,4.4,2,1
3,82.7,Luxury compact cars,2.93,7.40,4.93,15.88,4.37,4.17,39.25,4.25,...,4.18,NaN,-0.8,23.69,3.90,3.80,3.90,3.6,3,1
4,84.89,"Sports/sporty cars over $40,000",1.98,4.43,3.06,12.68,4.60,4.70,39.50,4.50,...,NaN,NaN,0.1,19.84,3.80,3.30,3.50,3.3,1,2


In [11]:
df_tests[['roadTestScore','duffel','suitcase']] = df_tests[['roadTestScore','duffel','suitcase']].astype(float)
df_tests['ratingsCategory'] = df_tests['ratingsCategory'].astype('category')
df_tests.describe()

,roadTestScore,0-30 Mph,0-60 Mph,45-65 Mph,"Quarter Mile,Sec",Shifting Smoothness,Overall Routine Handling,Turning Circle,Avoidance Confidence,Avoidance Speed,...,Controls And Displays For Tall,Merp,Points Off,Overall (Mpg),Interior Fabric,Other Cockpit,Primary Cockpit,Trunc,duffel,suitcase
count,255.000000,255.000000,255.000000,255.000000,255.000000,243.000000,255.000000,255.000000,255.000000,255.000000,...,179.000000,149.000000,219.000000,255.000000,255.000000,255.000000,255.000000,255.000000,132.000000,132.000000
mean,74.368549,3.092431,7.980471,5.152000,16.135765,4.304033,3.692196,39.366078,3.465137,51.965725,...,3.738268,31.485772,-0.457945,26.179255,3.805922,3.625725,3.754588,3.704000,2.265152,1.439394
std,11.201709,0.542229,1.608554,0.997353,1.277406,0.470583,0.556513,3.132566,0.700187,2.938864,...,0.818061,16.468477,0.523102,14.104585,0.677512,0.709768,0.699212,0.517682,0.931713,0.831237
min,19.950000,1.350000,3.480000,2.400000,12.100000,1.900000,1.920000,23.670000,1.250000,44.500000,...,1.040000,0.000000,-1.920000,14.100000,1.870000,1.500000,1.500000,2.200000,0.000000,0.000000
25%,67.800000,2.720000,6.875000,4.550000,15.375000,4.155000,3.370000,37.480000,3.000000,50.170000,...,3.390000,26.060000,-0.800000,20.090000,3.300000,3.100000,3.265000,3.400000,2.000000,1.000000
50%,75.840000,3.070000,7.850000,5.000000,16.100000,4.430000,3.700000,39.250000,3.600000,52.380000,...,3.900000,32.660000,-0.500000,23.400000,3.900000,3.600000,3.800000,3.700000,2.000000,1.000000
75%,81.710000,3.440000,8.930000,5.780000,16.930000,4.600000,4.065000,40.710000,3.925000,54.070000,...,4.320000,39.290000,0.000000,28.195000,4.400000,4.270000,4.300000,4.000000,3.000000,2.000000
max,99.880000,4.520000,12.080000,7.930000,19.010000,5.000000,4.970000,55.670000,5.000000,59.350000,...,5.000000,71.050000,0.500000,139.000000,5.000000,5.000000,5.000000,5.000000,4.000000,4.000000


In [12]:
list(df_tests['ratingsCategory'].unique())

['Sports/sporty cars under $40,000',
 'Luxury compact cars',
 'Sports/sporty cars over $40,000',
 'Midsized sport-utility vehicles',
 'Large cars',
 'Compact cars',
 'Ultra luxury cars',
 'Compact sport-utility vehicles',
 'Luxury midsized cars',
 'Roadsters',
 'Subcompact cars',
 'Midsized cars',
 'Mini-vans',
 'Large sport-utility vehicles',
 'Luxury large sport-utility vehicles',
 'Full-sized pickup trucks',
 'Compact pickup trucks',
 'Luxury midsized sport-utility vehicles',
 'Small cars (2-door)',
 'Electric cars/Plug-in hybrids',
 'Luxury compact sport-utility vehicles',
 'All-wheel drive Wagons',
 'Luxury entry-level cars',
 '4-seat Convertibles',
 'Luxury entry-level sport-utility vehicles',
 'Subcompact sport-utility vehicles']

In [13]:
cat=['Compact cars','Large cars','Midsized cars']
df_sedans=df_tests[df_tests['ratingsCategory'] == 'Subcompact cars']

for c in cat:
    df_sedans=df_sedans.append(df_tests[df_tests['ratingsCategory'] == c], ignore_index=True)
    
df_sedans=df_sedans.drop("Merp",axis=1)

In [14]:
df_sedans.columns.tolist()

['roadTestScore',
 'ratingsCategory',
 '0-30 Mph',
 '0-60 Mph',
 '45-65 Mph',
 'Quarter Mile,Sec',
 'Shifting Smoothness',
 'Overall Routine Handling',
 'Turning Circle',
 'Avoidance Confidence',
 'Avoidance Speed',
 'Dry Skid Lateralg',
 'Predictability Controllability',
 'Steering Feel',
 'Brake Fade',
 'Brake Modulation',
 'Dry Braking Directional Stability',
 'No-Lock Stopping Distance',
 'Split Mu Braking Directional Stability',
 'Split Mu Stopping Distance',
 'Wet Braking Stopping Distance',
 'Normal Load Highway',
 'Normal Load Rough Road1',
 'Normal Load Rough Road2',
 'Noise Trip Overall',
 'Driving Position For Medium',
 'Driving Position For Short',
 'Driving Position For Tall',
 'Driving Visibility For Medium',
 'Driving Visibility For Short',
 'Driving Visibility For Tall',
 'Front Seat Comfort For Medium',
 'Front Seat Comfort For Short',
 'Front Seat Comfort For Tall',
 'Rear Seat Comfort With 2',
 'Climate Control Overall',
 'Front Access For Medium',
 'Front Access For

In [15]:
sedanVals=df_sedans.drop("ratingsCategory",axis=1).values
reg = linear_model.LinearRegression()
reg.fit (sedanVals[:,1:],sedanVals[:,0])

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [25]:
df_1 = df[df['roadTestScore']!='None'].reset_index(drop='True')
df_accel=df_1[['Acceleration','0-30 Mph','0-60 Mph','45-65 Mph','Quarter Mile,Sec']]
df_accel[['Acceleration']] = df_accel[['Acceleration']].astype(float)

E:\python\WinPython\python-3.6.0.amd64\lib\site-packages\pandas\core\frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [26]:
df_accel.describe()

,Acceleration,0-30 Mph,0-60 Mph,45-65 Mph,"Quarter Mile,Sec"
count,255.000000,255.000000,255.000000,255.000000,255.000000
mean,3.764706,3.092431,7.980471,5.152000,16.135765
std,0.822619,0.542229,1.608554,0.997353,1.277406
min,2.000000,1.350000,3.480000,2.400000,12.100000
25%,3.000000,2.720000,6.875000,4.550000,15.375000
50%,4.000000,3.070000,7.850000,5.000000,16.100000
75%,4.000000,3.440000,8.930000,5.780000,16.930000
max,5.000000,4.520000,12.080000,7.930000,19.010000


In [27]:
accelVals=df_accel.values
reg = linear_model.LinearRegression()
reg.fit (accelVals[:,1:],accelVals[:,0])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [28]:
reg.coef_

array([ 0.13337909, -0.67568416, -0.06273858,  0.24904859])